# 2.3    元组比仅仅是不可变列表
有些Python入门教程把元组称为“不可变列表”，然而这并没有完全概括元组的特点。除了用作不可变的列表，它还可以用于没有字段名的记录。
鉴于后者常常被忽略，我们先来看看元组作为记录的功用。

## 2.3.1    元组和记录
####    示例 2-7　把元组用作记录

In [43]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


% 格式运算符能被匹配到对应的元组元素上。

In [44]:
for country, _ in traveler_ids:
    print(country)

USA
BRA
ESP


for 循环可以分别提取元组里的元素，也叫作拆包（unpacking）。因为元组中第二个元素对我们没有什么用，所以它赋值给“_”占位符。
## 2.3.2    元组拆包
示例 2 - 7 中，我们把元组('Tokyo', 2003, 32450, 0.66, 8014)里的元素分别赋值给变量city、year、pop、chg和area，而这所有的赋值我们只用一行声明就写完了。
同样，在后面一行中，一个 % 运算符就把passport元组里的元素对应到了 print函数的格式字符串空档中。这两个都是对元组拆包的应用。
最好辨认的元组拆包形式就是平行赋值，也就是说把一个可迭代对象里的元素，一并赋值到由对应的变量组成的元组中。就像下面这段代码

In [45]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates  # 元组拆包
latitude
longitude

-118.408056

另外一个很优雅的写法当属不使用中间变量交换两个变量的值：

In [46]:
b, a = a, b

还可以用 * 运算符把一个可迭代对象拆开作为函数的参数：

In [47]:
divmod(20, 8)

(2, 4)

In [48]:
t = (20, 8)
divmod(*t)

(2, 4)

In [49]:
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

下面是另一个例子，这里元组拆包的用法则是让一个函数可以用元组的形式返回多个值，
然后调用函数的代码就能轻松地接受这些返回值。比如os.path.split()函数就会返回以路径和最后一个文件名组成的元组(path, last_part):

In [50]:
import os
_, filename = os.path.split('/home/luciano/.ssh/idrsa.pub')
filename

'idrsa.pub'

In [51]:
_

'/home/luciano/.ssh'

在进行拆包的时候，我们不总是对元组里所有的数据都感兴趣，_
占位符能帮助处理这种情况，上面这段代码也展示了它的用法。
除此之外，在元组拆包中使用 * 也可以帮助我们把注意力集中在元组的部分元素上。
在Python中，函数用 * args来获取不确定数量的参数算是一种经典写法了。
于是Python3里，这个概念被扩展到了平行赋值中：

In [52]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [53]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [54]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

在平行赋值中，*前缀只能用在一个变量名前面，但是这个变量可以出现在赋值表达式的任意位置：

In [55]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

In [56]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

## 2.3.3    嵌套元组拆包
接受表达式的元组可以是嵌套式的，例如(a, b, (c, d))。
只要这个接受元组的嵌套结构符合表达式本身的嵌套结构，Python
就可以作出正确的对应。示例2 - 8就是对嵌套元组拆包的应用。
####    示例 2-8　用嵌套元组来获取经度

In [57]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),  # ➊
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas:  # ➋
    if longitude <= 0:  # ➌
        print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


元组已经设计得很好用了，但作为记录来用的话，还是少了一个功能：我们时常会需要给记录中的字段命名。namedtuple
函数的出现帮我们解决了这个问题
## 2.3.4    具名元组
collections.namedtuple是一个工厂函数，它可以用来构建一个带字段名的元组和一个有名字的类——这个带名字的类对调试程序有很大帮助。
####    示例 2-9 展示了如何用具名元组来记录一个城市的信息

In [58]:
from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [59]:
tokyo.population

36.933

In [60]:
tokyo.coordinates

(35.689722, 139.691667)

In [61]:
tokyo[1]

'JP'

创建一个具名元组需要两个参数，一个是类名，另一个是类的各个字段的名字。后者可以是由数个字符串组成的可迭代对象，或者是由空格分隔开的字段名组成的字符串。
存放在对应字段里的数据要以一串参数的形式传入到构造函数中（注意，元组的构造函数却只接受单一的可迭代对象）
你可以通过字段名或者位置来获取一个字段的信息。除了从普通元组那里继承来的属性之外，具名元组还有一些自己专有的属性。
示例2 - 10中就展示了几个最有用的：_fields类属性、类方法_make(iterable)和实例方法_asdict()。
####    示例 2-10　具名元组的属性和方法（接续前一个示例）

In [62]:
City._fields

('name', 'country', 'population', 'coordinates')

In [63]:
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': LatLong(lat=28.613889, long=77.208889)}

In [64]:
for key, value in delhi._asdict().items():
    print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: LatLong(lat=28.613889, long=77.208889)
